<a href="https://colab.research.google.com/github/dqtuan99/SlimCAE_PyTorch/blob/main/SlimCAE_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

There are many library conflicts in Colab due to preinstalled packages. My first setup cell locks compatible versions and then deliberately kills the kernel (os.kill(os.getpid(), 9)) to force a clean restart. This is expected. After the runtime reconnects, start from Cell 2. Any errors shown when the kernel is terminated can be ignored.

In [ ]:
# ⬇️ Run this cell first: pin NumPy/torch/core deps → auto-restart
import os, sys, subprocess

def run(*args):
    print(">>>", " ".join(args))
    subprocess.check_call(list(args))

# (1) Proactively uninstall packages that may pull in NumPy 2.x (ignore if missing)
for pkg in [
    "numpy","scipy","opencv-python","opencv-contrib-python","opencv-python-headless",
    "thinc","tsfresh","sentence-transformers","peft","transformers"
]:
    subprocess.call(["pip","uninstall","-y",pkg])

# (2) Pin the core versions
run("pip","install","--no-cache-dir","numpy==1.26.4","scipy==1.11.4")
run("pip","install","--no-cache-dir",
    "torch==2.1.2+cu118","torchvision==0.16.2+cu118","torchaudio==2.1.2+cu118",
    "--index-url","https://download.pytorch.org/whl/cu118")
run("pip","install","--no-cache-dir","pytorch-lightning==1.9.5","hydra-core","omegaconf")
run("pip","install","--no-cache-dir","pybind11","ninja","pybind11-stubgen==0.10.0")

# (3) CompressAI (incl. GDN deps) and MS-SSIM
run("pip","install","--no-cache-dir","compressai==1.2.6","pytorch-msssim==0.2.1")

# Show versions
import numpy, torch
print("NumPy:", numpy.__version__)
print("Torch:", torch.__version__)

print("\n🔁 Restarting runtime… (after reconnect, continue from Cell 2)")
os.kill(os.getpid(), 9)  # ← force-restart the Colab kernel


>>> pip install --no-cache-dir numpy==1.26.4 scipy==1.11.4
>>> pip install --no-cache-dir torch==2.1.2+cu118 torchvision==0.16.2+cu118 torchaudio==2.1.2+cu118 --index-url https://download.pytorch.org/whl/cu118
>>> pip install --no-cache-dir pytorch-lightning==1.9.5 hydra-core omegaconf
>>> pip install --no-cache-dir pybind11 ninja pybind11-stubgen==0.10.0
>>> pip install --no-cache-dir compressai==1.2.6 pytorch-msssim==0.2.1


In [ ]:
import os, sys, subprocess, shutil, site, inspect, pathlib

REPO = "/content/cbench_BaSIC"
URL  = "https://github.com/worldlife123/cbench_BaSIC.git"

def run(cmd, cwd=None, check=False):
    print(">>>", " ".join(cmd))
    p = subprocess.run(cmd, cwd=cwd, text=True, capture_output=True)
    # dump full logs
    if p.stdout:
        print(p.stdout)
    if p.returncode != 0:
        print("❌ STDERR:\n", p.stderr)
        if check:
            raise subprocess.CalledProcessError(p.returncode, cmd)
    return p.returncode == 0

# 0) Required packages (Cell 1 should have installed these; reinforce if missing)
run([sys.executable,"-m","pip","install","--no-cache-dir",
     "compressai==1.2.6","pytorch-msssim==0.2.1"])
run([sys.executable,"-m","pip","install","--no-cache-dir",
     "pybind11","ninja","pybind11-stubgen==0.10.0"])

# 1) Clean clone/reset
if not os.path.isdir(REPO):
    run(["git","clone",URL,REPO], check=True)
else:
    run(["git","fetch"], cwd=REPO, check=True)
    run(["git","reset","--hard","origin/main"], cwd=REPO, check=True)

# 2) Remove previous build artifacts + stale egg-link
for p in ["build","cbench.egg-info"]:
    shutil.rmtree(os.path.join(REPO,p), ignore_errors=True)

for sp in set(site.getsitepackages() + [site.getusersitepackages()]):
    egg = os.path.join(sp, "cbench.egg-link")
    if os.path.exists(egg):
        print("🧹 remove:", egg)
        os.remove(egg)

# 3) Editable install with build isolation disabled (important!)
ok = run([sys.executable,"-m","pip","install","--no-build-isolation","-e",REPO])

# 4) Fallback to setup.py if pip -e fails
if not ok:
    print("⚠️ pip -e failed → trying fallback: setup.py build develop")
    ok = run([sys.executable,"setup.py","build","develop"], cwd=REPO)

if not ok:
    raise SystemExit("❌ Install failed. Check STDERR above.")

# 5) Import test
try:
    import cbench
except ModuleNotFoundError:
    # Path fallback (in case only egg-link was created or nested layout)
    sys.path.insert(0, REPO)
    alt = os.path.join(REPO, "cbench_BaSIC")
    if os.path.isdir(alt):
        sys.path.insert(0, alt)
    import cbench

print("cbench from:", inspect.getfile(cbench))

from cbench.nn.layers.slimmable_layers import (
    DynamicConv2d, DynamicGDN,
    DynamicResidualBlock, DynamicResidualBlockUpsample,
)
print("✅ Layers OK:", DynamicConv2d, DynamicGDN)


>>> /usr/bin/python3 -m pip install --no-cache-dir compressai==1.2.6 pytorch-msssim==0.2.1

>>> /usr/bin/python3 -m pip install --no-cache-dir pybind11 ninja pybind11-stubgen==0.10.0

>>> git clone https://github.com/worldlife123/cbench_BaSIC.git /content/cbench_BaSIC
>>> /usr/bin/python3 -m pip install --no-build-isolation -e /content/cbench_BaSIC
Obtaining file:///content/cbench_BaSIC
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'

❌ STDERR:
   error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.

⚠️ pip -e failed → trying fallback: setup.py build 

In [ ]:
# slimcae_wrapper.py
import torch
import torch.nn as nn
from cbench.nn.layers.slimmable_layers import (
    DynamicConv2d, DynamicGDN,
    DynamicResidualBlock, DynamicResidualBlockUpsample
)

class SlimCAE(nn.Module):
    def __init__(self, channels_list=(64, 96, 128)):
        super().__init__()
        C = list(channels_list)            # e.g., [64, 96, 128]
        C_out3 = [3] * len(C)              # IMPORTANT: same length as the number of slim levels

        # Encoder
        self.enc = nn.Sequential(
            DynamicConv2d(3, C, 5, 2, None, bias=True),
            DynamicGDN(C),
            DynamicResidualBlock(C[-1], C),
            DynamicConv2d(C, C, 5, 2, None, bias=True),
            DynamicGDN(C),
        )

        # Decoder
        self.dec = nn.Sequential(
            DynamicResidualBlockUpsample(C[-1], C, upsample=2),
            DynamicResidualBlock(C[-1], C),
            DynamicConv2d(C, C_out3, 5, 1, None, bias=True),  # NOTE: out-channels must be a list here
        )

    def set_level(self, level: int):
        # Propagate the selected slim level to all dynamic modules
        for m in self.modules():
            if hasattr(m, "set_complex_level"):
                m.set_complex_level(level)

    def forward(self, x):
        z = self.enc(x)
        y = self.dec(z)
        return y, z



In [ ]:
net = SlimCAE((64,96,128))
net.set_level(1)
out, z = net(torch.randn(1,3,256,256))
print(out.shape)

torch.Size([1, 3, 128, 128])


Todo: Rate Distortion Complexity Trade Off